# Fast Feature Extraction With Data Augmentation

Is it a triangle, circle, or square?

https://www.kaggle.com/cactus3/basicshapes/home
28x28

     There are two parts of an imagine classification network:
     it begins witn a series of pooling and convolution layers, then ends with a densely connect classifier
     The convolutional base of a model is the first part
     pg. 143 - 144
     
    

### Initialize convolutional base

In [7]:
from keras.applications.mobilenetv2 import MobileNetV2


conv_base = MobileNetV2(
    weights='imagenet',
    input_shape=(224,224, 3),
    include_top=False # exclude the densely connected classifer, which sits on top of hte convolutional network
)
conv_base.trainable = False
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

### Add a densely connected classifier on top of the convolutional base

In [9]:
from keras import models, layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16056576  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 18,314,817
Trainable params: 16,056,833
Non-trainable params: 2,257,984
_________________________________________________________________


### Load samples into Pandas DataFrame

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# initialize model

# augment training data to produce a model capable of handling data variations
train_datagen = ImageDataGenerator(
    rescale=1./255, # rescale to target values between 0 and 255 (default between 0 and 1)
    rotation_range=40, # train on variations rotated up to 40 degrees
    width_shift_range=0.9, # train using variations off-center on x-axis by factor of 0.9
    height_shift_range=0.9, # train using variations off-center on y-axis by a factor of 0.9
    shear_range=0.2, # train using variations sheared/warped by a factor of 0.2
    zoom_range=0.9, # train using variations zoomed by a factor of 0.9
    horizontal_flip=True, # x-axis flip
    vertical_flip=True # y-axis flip
)

test_datagen = ImageDataGenerator(
    rescale=1./255, # rescale to target values between 0 and 255 (default between 0 and 1)
)

# walk through './data/shapes' and load filenames into a dataframe with labels

root, dirs, files = next(os.walk('./data/shapes/'))

samples_df = pd.DataFrame([
    {
        'label': file.split('_')[0], # filename format '<label>_<int>.png'
        'filename': file
    } for file in files
])
# samples_df = pd.concat([square_sample_df, circle_sample_df, triangle_sample_df])
# samples_df.reset_index(inplace=True, drop=True)


samples_df


Using TensorFlow backend.
/home/pi/rpi-vision/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/home/pi/rpi-vision/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


,filename,label
0,triangle_18.png,triangle
1,triangle_43.png,triangle
2,square_46.png,square
3,square_24.png,square
4,triangle_4.png,triangle
5,square_1.png,square
6,triangle_63.png,triangle
7,circle_72.png,circle
8,circle_71.png,circle
9,triangle_44.png,triangle


In [5]:
train_df, validation_df = train_test_split(samples_df, test_size=0.25)

train_df.reset_index(inplace=True, drop=True)
validation_df.reset_index(inplace=True, drop=True)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./data/shapes/',
    batch_size=len(train_df),
    class_mode='binary', # use binary labels for binary_crossentropy loss calculations
    target_size=(244,244),
    y_col='label',
)


validation_generator = test_datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory='./data/shapes/',
    batch_size=len(validation_df),
    class_mode='binary', # use binary labels for binary_crossentropy loss calculations
    target_size=(244,244),
    y_col='label',
    classes=['square', 'circle', 'triangle']

)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5), # what is this?
    metrics=['acc'] # accuracy
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)


['circle_1.png', 'circle_10.png', 'circle_100.png', 'circle_11.png', 'circle_12.png', 'circle_13.png', 'circle_14.png', 'circle_15.png', 'circle_16.png', 'circle_17.png', 'circle_18.png', 'circle_19.png', 'circle_2.png', 'circle_20.png', 'circle_21.png', 'circle_22.png', 'circle_23.png', 'circle_24.png', 'circle_25.png', 'circle_26.png', 'circle_27.png', 'circle_28.png', 'circle_29.png', 'circle_3.png', 'circle_30.png', 'circle_31.png', 'circle_32.png', 'circle_33.png', 'circle_34.png', 'circle_35.png', 'circle_36.png', 'circle_37.png', 'circle_38.png', 'circle_39.png', 'circle_4.png', 'circle_40.png', 'circle_41.png', 'circle_42.png', 'circle_43.png', 'circle_44.png', 'circle_45.png', 'circle_46.png', 'circle_47.png', 'circle_48.png', 'circle_49.png', 'circle_5.png', 'circle_50.png', 'circle_51.png', 'circle_52.png', 'circle_53.png', 'circle_54.png', 'circle_55.png', 'circle_56.png', 'circle_57.png', 'circle_58.png', 'circle_59.png', 'circle_6.png', 'circle_60.png', 'circle_61.png', '

NameError: name 'model' is not defined

In [ ]:
import mathplotlib.pyplot as plt